In [6]:
from kafka import KafkaProducer
import threading
import json
import random
from time import sleep

from equipo.random_tweets import generate_random_tweets  

KAFKA_SERVER = 'd9000434c2b2:9093'
TOPIC_PREFIX = 'tweet-'

In [7]:
def worker(user_id: int, topic: str, num_messages: int):

    # Initialize the Kafka producer
    producer = KafkaProducer(
        bootstrap_servers=KAFKA_SERVER,
        value_serializer=lambda v: json.dumps(v).encode('utf-8')  # serialize data as JSON
    )

    print(f"[Thread-{user_id}] Iniciando, topic={topic}, mensajes={num_messages}")
    
    # Produce data to Kafka topic
    try:
        for tweet in generate_random_tweets(num_messages, user_id):
            # Send data to Kafka
            producer.send(topic, tweet)
            print(f"[Thread-{user_id}] Sent to {topic}: {tweet['tweet_id']}")

            sleep(3)

    except KeyboardInterrupt:
        print(f"[Thread-{user_id}] Interrupted.")
        
    finally:
        producer.close()
        print(f"[Thread-{user_id}] Producer closed.")

In [8]:
if __name__ == "__main__":
    NUM_MESSAGES = random.randint(8, 12) #500, 501
    threads = []

    for user_id in range(1, 5):
        topic = f"{TOPIC_PREFIX}{user_id}"
        t = threading.Thread(
            target=worker,
            args=(user_id, topic, NUM_MESSAGES),
            name=f"Thread-{user_id}"
        )
        threads.append(t)
        t.start()

    # Espera a que todos los hilos terminen
    for t in threads:
        t.join()

    print("Finished.")


[Thread-1] Iniciando, topic=tweet-1, mensajes=9
[Thread-2] Iniciando, topic=tweet-2, mensajes=9
[Thread-4] Iniciando, topic=tweet-4, mensajes=9
[Thread-3] Iniciando, topic=tweet-3, mensajes=9
[Thread-2] Sent to tweet-2: f0408743-3b92-4bf7-8212-8270198a614c
[Thread-3] Sent to tweet-3: 30ef3d77-20b7-4534-8f98-cafd07a631f2
[Thread-4] Sent to tweet-4: 848b3af3-1062-4e21-afdf-863b228ed3d4
[Thread-1] Sent to tweet-1: 315611ef-1b6c-4205-8da9-011d2736060c
[Thread-2] Sent to tweet-2: 68006eb6-0a3d-4162-896c-21c516927574
[Thread-4] Sent to tweet-4: 75b5c7c6-8498-4176-9107-576fb986feab
[Thread-3] Sent to tweet-3: 042d47ab-e9b9-4ba9-93fb-f4aa44d2a526
[Thread-1] Sent to tweet-1: cb514a24-ede0-48ee-ab9f-f8c722629f39
[Thread-2] Sent to tweet-2: 57195393-5166-4004-8c78-df5a6bf86a5d
[Thread-4] Sent to tweet-4: d554fb57-7147-4364-8c26-ebf10724f4e4
[Thread-3] Sent to tweet-3: ac267587-8ca0-42fd-b9fd-b063f7eb71da
[Thread-1] Sent to tweet-1: 2acc9f35-b295-4c52-9819-3e7fb7b025a5
[Thread-2] Sent to tweet-2: 